# Preprocess subject data

In [1]:
import sys
import os
from pathlib import Path
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
from src.configs.config import CFGLog
from src.preprocessing.utils import get_time_window


imported main module
imported preprocessing module


KeyboardInterrupt: 

Load in subject IDs and paths from CFGLog

In [ ]:
config = Config.from_json(CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC
Panc_subject_ids = config.data.chronic_pancreatitis.subject_ids.CP
Lupus_CP_subject_ids = config.data.lupus.subject_ids.CP
Lupus_NP_subject_ids = config.data.lupus.subject_ids.NP

Create paths

In [ ]:
# Data pahts
abs_path = Path(os.path.dirname(os.path.abspath("../../../")))

CLBP_path = abs_path / config.data.chronic_low_back_pain.path
Panc_path = abs_path / config.data.chronic_pancreatitis.path
Lupus_path = abs_path / config.data.lupus.path

# Save paths
times_tup, time_win_path = get_time_window(5)
save_path = Path("../../data/preprocessed/")
stc_path = save_path / "source_time_courses/"
eyes_open_path = stc_path / "eyes_open"
evoked_path = stc_path / "evoked" / time_win_path

paths = [save_path, stc_path, eyes_open_path, evoked_path]
[os.makedirs(path, exist_ok=True) for path in paths];

Define preprocessing parameters

In [ ]:
# Time range
time_range=(-0.2, 0.8)

# Baseline
baseline=(-2.5, 0)

Combine subject IDs

In [ ]:
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids,
    "chronic_pancreatitis": Panc_subject_ids,
    "lupus": Lupus_CP_subject_ids + Lupus_NP_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain']) + len(sub_ids['chronic_pancreatitis']) + len(sub_ids['lupus'])}")
print(sub_ids)

Create Subject object for each subject

In [ ]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        print(subject)
        
        subject.load_raw()
        subject.get_cleaned_resting()
        
        break

In [ ]:
# Create an instance of SubjectProcessor
processor = SubjectProcessor(paths_dict, roi_acronyms)

# Loop through each subject and process the response
for sub_id in sub_ids_focus:
    subject = Subject(sub_id)
    processor.plot_TFR_and_trace(subject,
                                 channel=channel, 
                                 baseline=baseline,
                                 time_range=time_range,
                                 vlim=vlim,
                                 )
    # processor.process_response(subject)
    # break
    

# Display the results
processor.display_results()
